In [4]:
import pandas as pd
import numpy as np
import time
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import dump

In [16]:
ratings_df = pd.read_csv("ratings_data.csv")
ratings_df.head(5)

,userId,movieId,title,rating
0,1,1371,Rocky III,2.5
1,1,1405,Greed,1.0
2,1,2105,American Pie,4.0
3,1,2193,My Tutor,2.0
4,1,2294,Jay and Silent Bob Strike Back,2.0


In [11]:
svd = dump.load('svd_model')
svd_algo = svd[1]

In [5]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainDataset = Dataset.load_from_df(training_dataframe[['userId', 'movieId', 'rating']], reader)
    testDataset = Dataset.load_from_df(testing_dataframe[['userId', 'movieId', 'rating']], reader)
    trainset = trainDataset.build_full_trainset()
    testset = (testDataset.build_full_trainset()).build_testset()
    return trainDataset, testDataset, trainset, testset

In [6]:
file_path_train = 'training_data.csv'
file_path_test = 'testing_data.csv'
traindf = pd.read_csv(file_path_train)
testdf = pd.read_csv(file_path_test)
trainDataset, testDataset, trainset, testset = convert_traintest_dataframe_forsurprise(traindf, testdf)

In [20]:
Mapping_file = dict(zip(ratings_df['title'].tolist(), ratings_df['movieId'].tolist()))

In [25]:
def pred_user_rating(ui):
    if ui in ratings_df.userId.unique():
        ui_list = ratings_df[ratings_df.userId == ui].movieId.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = svd_algo.predict(ui, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("User Id does not exist in the list!")
        return None

In [26]:
user_id = 1
pred_user_rating(user_id)

,ratings
movies,
Galaxy Quest,4.279369
Sleepless in Seattle,4.273430
The Million Dollar Hotel,4.252846
The Thomas Crown Affair,4.222947
Nell,4.206096
Pandora's Box,4.194293
The Good Thief,4.164970
The Green Mile,4.106310
Madagascar,4.082344


In [30]:
user_1 = ratings_df[ratings_df['userId'] == 1]
user_1

,userId,movieId,title,rating
0,1,1371,Rocky III,2.5
1,1,1405,Greed,1.0
2,1,2105,American Pie,4.0
3,1,2193,My Tutor,2.0
4,1,2294,Jay and Silent Bob Strike Back,2.0
